In [ ]:
%pip install 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# Check .env API key

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
env_path = Path('/home/wong/coding/thesis/') / '.env'
load_dotenv(dotenv_path=env_path, verbose=True)

# Verify that essential environment variables are loaded
print("Environment variables loaded:")
if os.environ.get('GOOGLE_API_KEY'):
    print("✓ GOOGLE_API_KEY is set")
else:
    print("✗ GOOGLE_API_KEY is not set")

if os.environ.get('LANGCHAIN_API_KEY'):
    print("✓ LANGCHAIN_API_KEY is set")
else:
    print("✗ LANGCHAIN_API_KEY is not set")

In [ ]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

retriever = vectorstore.as_retriever()